In [1]:
from langchain_community.document_loaders import PyPDFLoader
import os

pdf_path = os.path.join("../bot1", "data", "monopoly.pdf") 

loader = PyPDFLoader(pdf_path)
docs = loader.load()
docs

[Document(metadata={'source': '../bot1\\data\\monopoly.pdf', 'page': 0}, page_content='MONOPOLY \nProperty Trading Game from Parker Brothers" \nAGES 8+ \n2 to 8 Players \nContents: Gameboard, 3 dice, tokens, 32 houses, I2 hotels, Chance \nand Community Chest cards, Title Deed cards, play money and a Banker\'s tray. \nNow there\'s a faster way to play MONOPOLY. Choose to play by \nthe classic rules for buying, renting and selling properties or use the \nSpeed Die to get into the action faster. If you\'ve never played the classic \nMONOPOLY game, refer to the Classic Rules beginning on the next page. \nIf you already know how to play and want to use the Speed Die, just \nread the section below for the additional Speed Die rules. \nSPEED DIE RULES \nLearnins how to Play with the S~eed Die IS as \n/ \n fast as playing with i\'t. \n1. When starting the game, hand out an extra $1,000 to each player \n(two $5005 should work). The game moves fast and you\'ll need \nthe extra cash to buy and bu

In [9]:
# os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")



In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)    # 1000 --->500
# documents=text_splitter.split_documents(docs)
documents=text_splitter.split_documents(docs)
documents[:5]

# making the chunks of the data with overlap and splitting each chunks 


[Document(metadata={'source': '../bot1\\data\\monopoly.pdf', 'page': 0}, page_content='MONOPOLY \nProperty Trading Game from Parker Brothers" \nAGES 8+ \n2 to 8 Players \nContents: Gameboard, 3 dice, tokens, 32 houses, I2 hotels, Chance \nand Community Chest cards, Title Deed cards, play money and a Banker\'s tray. \nNow there\'s a faster way to play MONOPOLY. Choose to play by \nthe classic rules for buying, renting and selling properties or use the \nSpeed Die to get into the action faster. If you\'ve never played the classic \nMONOPOLY game, refer to the Classic Rules beginning on the next page. \nIf you already know how to play and want to use the Speed Die, just \nread the section below for the additional Speed Die rules. \nSPEED DIE RULES \nLearnins how to Play with the S~eed Die IS as \n/ \n fast as playing with i\'t. \n1. When starting the game, hand out an extra $1,000 to each player \n(two $5005 should work). The game moves fast and you\'ll need \nthe extra cash to buy and bu

# Open AI Embeddings

In [3]:
from sentence_transformers import SentenceTransformer
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
model = "sentence-transformers/all-mpnet-base-v2"

hf = HuggingFaceEmbeddings(
    model_name = model,
    model_kwargs = {'device' : 'cpu'},
    encode_kwargs = {'normalize_embeddings' : False},
)

e:\sushant\Jupyter Notebook\Intern_Project\LnagChain_chatbot\RAG Chatbot\myenv\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
C:\Users\HP\AppData\Local\Temp\ipykernel_4472\2898794908.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  hf = HuggingFaceEmbeddings(


In [4]:
# making the index of faiss
import faiss
index = faiss.IndexFlatL2(len(hf.embed_query("Hello World")))

In [6]:
from langchain_community.docstore.in_memory import InMemoryDocstore

# vector store combinding the index of faiss and embedding form our model
vector_store = FAISS(
    embedding_function = hf,
    index = index,
    docstore = InMemoryDocstore(),
    index_to_docstore_id = {},
)

## FAISS Vector Database - using OPENAI embeddings
# just making and storing the first fifteen chunks of the data 

vector_store.add_documents(documents=documents)

['d9a750ab-0222-4460-a148-7dd4f999f62e',
 '2904fa9d-832e-40ad-ae88-0abdf5bbd193',
 '1047d1d1-4ed0-4381-86ba-99302c78ebc2',
 '44f6e5ae-d98b-424e-89e4-a4c75a221b3a',
 '04a02481-b709-4354-b597-4bf02308944f',
 'cb494db7-9335-4900-9071-2328e1351b3e',
 '71190197-79d7-4db1-b492-8392790cd2a3',
 '1807b0a8-a478-47fb-96d3-36ce6cf4acc8',
 '1f54d4bd-448c-4d61-a787-7b05388b28f3',
 '0b21d6bf-d5a7-43fd-a274-acc913d6f297',
 '101823bc-4de7-4210-a795-c0ef8a3524a2',
 'e9343966-0198-43df-ae8d-9c081f4f78ff',
 '87f7013f-1e61-4a06-bbd4-480e5a8c7ffb',
 '6fe83d7f-6aa7-4a24-af79-273e6466689c',
 'fe71b5e3-3b83-4b47-a8ac-5a099ebdb95a',
 'e025e541-22fe-41cf-8b87-58fdbb2c736d',
 '848fcb2a-ece7-434e-ac47-ed07e3f9523f',
 '531dad81-ea2e-4945-813f-e79cea17c931',
 'ec8c3aa9-7a8d-42fb-841d-c2eeda311146',
 '34fa267e-b239-4866-9be4-7895939e8a82',
 '6279aebd-887d-42ae-ab96-eabd43bb30e9',
 '5a3c2153-0d5f-47a2-ad28-7f499e4ddab7',
 '1a854f12-9b6c-4a88-9770-de8c619e1ce7']

In [10]:
from langchain.embeddings import OpenAIEmbeddings  # or your specific embedding model

# Initialize the embedding model
embeddings = OpenAIEmbeddings()  # Replace with the actual embedding class you used earlier

# Save the vector store
vector_store.save_local("FAISS_STORE1")

# Load the vector store with embeddings and allow dangerous deserialization
loaded_vector_store = vector_store.load_local(
    "FAISS_STORE", embeddings=embeddings, allow_dangerous_deserialization=True
)


In [11]:
index

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x0000014B39F5B300> >

In [12]:
from langchain_community import Ollama

# Load Ollama LLAMA2 model
llm = Ollama(model = "llama2")
llm

ImportError: cannot import name 'Ollama' from 'langchain_community' (e:\sushant\Jupyter Notebook\Intern_Project\LnagChain_chatbot\RAG Chatbot\myenv\lib\site-packages\langchain_community\__init__.py)

In [20]:
from langchain.llms import Ollama

# Initialize the LLM model (replace with the appropriate model name if different)
llm = Ollama(model="tinyllama:latest")  # Or "tinyllama:latest" if it exists

# Test the model with a sample prompt
try:
    response = llm("Hello, world!")  # Input your test prompt here
    print("Response:", response)
except Exception as e:
    print("Error:", e)


Response: Hello, everybody! This is the latest installment of Hello World!, our monthly blog series celebrating all things tech. As we kick off the new year, let's take a look back at what made the past few months so special and forward to what's to come in 2021.

Let's start with the big one: Apple released their latest product lineup on September 15th, and it was nothing short of spectacular. The new iPhone 12 and 12 mini were sleek, stylish, and packed with all of the latest features and technologies. I spent a few days using the iPhone 12 Pro Max for my own personal reasons and can say that they truly delivered on their promises to enhance your day-to-day experiences.

One of the standout features of the iPhone 12 is its new camera system, which boasts a 48-megapixel triple-lens setup on both models. The 12 mini has a 5-megapixel telephoto lens and an 11-megapixel ultra-wide lens. Both phones come with the latest A14 Bionic chip, which is one of the fastest mobile processors on the

In [22]:
llm

Ollama(model='tinyllama:latest')

# design Chatbot Template

In [ ]:
# from langchain_core.prompts import ChatPromptTemplate

# prompt = ChatPromptTemplate.from_template("""
#                                           Answer the following question based only on the provided context. 
#                                           Think step by step before providing a detailed answer.
#                                           <context>
#                                           {context} 
#                                           </context>
#                                           Question:{input}
#                                     """)

In [23]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""
You are a helpful assistant that answers questions based on the provided context. Follow these steps:

1. **Carefully read the provided context**: Understand the key points and any details that may be relevant to the question being asked.
2. **Analyze the question**: Think about how the context and the question relate to each other.
3. **Break down your answer logically**: Use the context to form a structured response, step by step, to ensure accuracy.
4. **Provide a detailed explanation**: Explain your reasoning clearly and comprehensively, ensuring that the answer is well-supported by the context.

Context:
{context}

Question:
{input}

Instructions:
- Be sure to base your answer only on the provided context, even if you have prior knowledge on the subject.
- If the context doesn't provide enough information to answer the question, clearly mention that.
- Your answer should be detailed and well-reasoned, breaking down each step of your thought process.

Please provide your detailed, step-by-step answer below.
""")


# Chain Introduction

In [24]:
# create stuff document chain

from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(llm, prompt)

A retriever is an interface that returns documents given an unstructured query. It is more general than a vector store. A retriever does not need to be able to store documents, only to return (or retrieve) them. Vector stores can be used as the backbone of a retriever, but there are other types of retrievers as well.

Retrievers accept a string query as input and return a list of Document's as output.

In [26]:
retriver = vector_store.as_retriever()
retriver

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000014B003ACDC0>)

*combine retriver and document chain to form a Retrival Chain*

A retrieval chain is a method for answering questions by combining a retriever with a document chain. The retriever's job is to extract the content of the most recent message from a list of input messages and pass it to the document chain, which then generates a response. 

In [27]:
# retriver chain

from langchain.chains import  create_retrieval_chain


retrival_chain = create_retrieval_chain(retriver, document_chain)

In [30]:
response = retrival_chain.invoke({"input":"How to play monopoly?"})

In [31]:
response["answer"]

"To play Monopoly, follow these instructions:\n\n1. Start by handing out an extra $1,000 to each player (two $5005 should work). The game moves fast and you'll need 2 white dice to move a value of one or 5 spaces.\n\n2. Mr. Monopoly is the only property left in the bank and there are no more properties in the bank. If you don't want to buy any more properties, move to the space anyway and put the property up for auction (you owe another player money).\n\n3. The Speed Die is used to determine if you rolled doubles, as described above. If you rolled doubles three times in a row, you can move anywhere you want on the board! Otherwise, do not look at the Speed Die.\n\n4. Each player moves their token through the board and buys properties one space at a time with money from the banker. The Banker collects all taxes, fine, loan, and interest payments.\n\n5. When you land on or pass over another player's property in the bank, they owe you money (the price of their property). If you don't want

In [32]:
response

{'input': 'How to play monopoly?',
 'context': [Document(metadata={'source': '../bot1\\data\\monopoly.pdf', 'page': 0}, page_content='MONOPOLY \nProperty Trading Game from Parker Brothers" \nAGES 8+ \n2 to 8 Players \nContents: Gameboard, 3 dice, tokens, 32 houses, I2 hotels, Chance \nand Community Chest cards, Title Deed cards, play money and a Banker\'s tray. \nNow there\'s a faster way to play MONOPOLY. Choose to play by \nthe classic rules for buying, renting and selling properties or use the \nSpeed Die to get into the action faster. If you\'ve never played the classic \nMONOPOLY game, refer to the Classic Rules beginning on the next page. \nIf you already know how to play and want to use the Speed Die, just \nread the section below for the additional Speed Die rules. \nSPEED DIE RULES \nLearnins how to Play with the S~eed Die IS as \n/ \n fast as playing with i\'t. \n1. When starting the game, hand out an extra $1,000 to each player \n(two $5005 should work). The game moves fast 